<a href="https://colab.research.google.com/github/kiranbeethoju/Stock-Market-Forecast/blob/master/stock_predictions_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
ticks_dict = [{"comapny":"AXISBANK.NS","start":"1999-01-01"}, {"comapny":"WIPRO.NS","start":"1996-01-01"},
              {"comapny":"HINDPETRO.NS","start":"1996-01-01"},
              {"comapny":"ONGC.NS","start":"1996-01-01"},
              {"comapny":"INFY.NS","start":"1996-01-01"},
              {"comapny":"TECHM.NS","start":"2006-08-01"},
              {"comapny":"TATACONSUM.NS","start":"2017-06-01"},
              {"comapny":"M&M.NS","start":"1996-01-01"}              ]



In [49]:
!pip install fbprophet
import datetime
import pandas as pd
import numpy as np
import pylab as pl
import datetime
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from matplotlib.collections import LineCollection
from pandas_datareader import data as wb
from sklearn import cluster, covariance, manifold

In [64]:
start = '1996-01-01'
end = '2020-07-15'
#edit your company here
tickers = ['HDFCBANK.NS']

thelen = len(tickers)

price_data = []
for ticker in tickers:
    prices = wb.DataReader(ticker, start = start, end = end, data_source='yahoo')[['Open','Adj Close']]
    price_data.append(prices.assign(ticker=ticker)[['ticker', 'Open', 'Adj Close']])

#names = np.reshape(price_data, (len(price_data), 1))

names = pd.concat(price_data)
names.reset_index()

#pd.set_option('display.max_columns', 500)

open = np.array([q['Open'] for q in price_data]).astype(np.float)
close = np.array([q['Adj Close'] for q in price_data]).astype(np.float)
print(len(names))
names = names.reset_index()
open = names.rename(columns={'Date':'ds','Open':'y'})
open = open[['ds','y']]
close = names.rename(columns={'Date':'ds','Adj Close':'y'})
close = close[["ds","y"]]
from fbprophet import Prophet
from plotly.graph_objs import *

df = open

def run(df,name):
  df = df[6000:]
  #df = df[df['ds'].dt.dayofweek < 5]
  m = Prophet()
  m.fit(df)
  future = m.make_future_dataframe(periods=400)
  future.tail()
  forecast = m.predict(future)
  future = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
  future = future.rename(columns={"yhat":"yhat_future","yhat_lower":"yhat_lower_future","yhat_upper":"yhat_upper_future"})
  mixed = pd.merge(df,future,on="ds",how="inner")
  mixed.tail(30)
  avg = (mixed["yhat_future"].iloc[0]+mixed["yhat_lower_future"].iloc[0]+mixed["yhat_upper_future"].iloc[0])/3
  mixed["avg_val"] = ''
  for i in range(0, len(mixed)):
    avg = (mixed["yhat_future"].iloc[i]+mixed["yhat_lower_future"].iloc[i]+mixed["yhat_upper_future"].iloc[i])/3
    mixed['avg_val'].iloc[i] = avg
  mixed = mixed
  import plotly as py

  import plotly.graph_objects as go
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["yhat_future"],
                      mode='lines+markers',
                      name='Predicted Opening Values'))
  fig.show()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["avg_val"],
                      mode='markers',
                      name='Predicted Opening Avg Values '))

  fig.show()
  future.to_csv(f'{name}-{tickers[0]}.csv')
  return mixed
a = run(open,"open")
b = run(close,"close")
a = a.rename(columns={"avg_val":"pred_open_avg","y":"actual_open","yhat_upper_future":"yhat_upper_future_open","yhat_future":"yhat_future_open","yhat_lower_future":"yhat_lower_future_open"})
b = b.rename(columns={"avg_val":"pred_close_avg","y":"actual_close","yhat_upper_future":"yhat_upper_future_close","yhat_future":"yhat_future_close","yhat_lower_future":"yhat_lower_future_close"})
c = pd.merge(a,b,on="ds",how="inner")
df = c[["ds","actual_open","pred_open_avg","actual_close","pred_close_avg"]]
df["open_diff"] = df["pred_open_avg"]-df["actual_open"]
df["close_diff"] = df["pred_close_avg"]-df["actual_close"]
df["actual_raise_of_value"] = df["actual_close"] - df["actual_open"]
df["predicted_raise_of_value"] = df["pred_close_avg"] - df["pred_open_avg"]
df.tail(10)
df.to_csv("{}_complete_analysis.csv".format(tickers[0]))

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


6055


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:78: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [65]:
df = pd.read_csv("{}_complete_analysis.csv".format(tickers[0]))
df.head()

,Unnamed: 0,ds,actual_open,pred_open_avg,actual_close,pred_close_avg,open_diff,close_diff,actual_raise_of_value,predicted_raise_of_value
0,0,2020-04-22,919.150024,941.136393,928.599976,955.723552,21.986369,27.123576,9.449951,14.587159
1,1,2020-04-23,935.000000,948.124397,954.950012,955.366908,13.124397,0.416896,19.950012,7.242511
2,2,2020-04-24,933.000000,935.063520,938.049988,947.850323,2.063520,9.800335,5.049988,12.786803
3,3,2020-04-27,945.150024,949.415754,929.700012,937.610446,4.265730,7.910433,-15.450012,-11.805309
4,4,2020-04-28,943.799988,933.360488,931.400024,926.507322,-10.439499,-4.892702,-12.399963,-6.853166


In [66]:
import plotly as py

import plotly.graph_objects as go
fig = go.Figure()
mixed = df
fig.update_layout(
      title="Actual Open vs Actual Close & Pred Open vs Pred Close Prices Trend of =>> {}".format(tickers[0]),
      xaxis_title="Date",
      yaxis_title="Price",
      font=dict(
          family="Courier New, monospace",
          size=16,
          color="#7f7f7f"
      )
  )

fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["actual_open"],
                    mode='lines',
                    name='Actual Opening Values',line=dict(color='mediumspringgreen', width=4, dash='solid')))
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["pred_open_avg"],
                    mode='lines+markers',
                    name='Predicted Opening Values',line=dict(color='mediumseagreen', width=4, dash='dot')))


fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["actual_close"],
                    mode='lines',
                    name='Actual Close Values ',line=dict(color='blue', width=4, dash='solid')))
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["pred_close_avg"],
                    mode='lines',
                    name='Predicted Close Values ',line=dict(color='royalblue', width=4, dash='dot')))
fig.show()
#### Drawing difference between actual vs predicted 


In [67]:
fig.update_layout(
      title="Actual Raise/Fall & Predicted Raise/Fall share value Trend of =>> {}".format(tickers[0]),
      xaxis_title="Date",
      yaxis_title="Price",
      font=dict(
          family="Courier New, monospace",
          size=16,
          color="#7f7f7f"
      )
  )
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["actual_raise_of_value"],
                    mode='lines',
                    name='Actual Opening Values',line=dict(color='mediumspringgreen', width=4, dash='solid')))
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["predicted_raise_of_value"],
                    mode='lines+markers',
                    name='Predicted Opening Values',line=dict(color='mediumseagreen', width=4, dash='dot')))
fig.show()